<a href="https://colab.research.google.com/github/Varsha-commuri/Projects/blob/main/skin_lesion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

from google.colab import files
uploaded = files.upload()   # upload the ZIP file here



Saving Monkeypox Skin Image Dataset.zip to Monkeypox Skin Image Dataset.zip


In [5]:
import zipfile
import os

# Get uploaded file name
zip_file_path = next(iter(uploaded))

# Extract to /content/
extract_dir = "/content/Monkeypox_Skin_Image_Dataset"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Check if folders extracted correctly
print(os.listdir(extract_dir))


['Monkeypox Skin Image Dataset']


In [6]:
# Path inside extracted folder
dataset_path = os.path.join(extract_dir, "Monkeypox Skin Image Dataset")

# Now check contents
print(os.listdir(dataset_path))


['Measles', 'Monkeypox', 'Chickenpox', 'Normal']


In [14]:
import tensorflow as tf

dataset_path = "/content/Monkeypox_Skin_Image_Dataset/Monkeypox Skin Image Dataset"
img_size = (224, 224)
batch_size = 32

# Training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    labels="inferred",
    label_mode="categorical",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    validation_split=0.2,
    subset="training",
    seed=123
)

# Save class names here BEFORE normalization
class_names = train_ds.class_names
print("Class names:", class_names)

# Validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    labels="inferred",
    label_mode="categorical",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    validation_split=0.2,
    subset="validation",
    seed=123
)


Found 770 files belonging to 4 classes.
Using 616 files for training.
Class names: ['Chickenpox', 'Measles', 'Monkeypox', 'Normal']
Found 770 files belonging to 4 classes.
Using 154 files for validation.


In [15]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [16]:
from tensorflow.keras import layers, models

num_classes = len(class_names)

model = models.Sequential([
    layers.InputLayer(input_shape=(224,224,3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds, validation_data=val_ds, epochs=5)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.3393 - loss: 7.1125 - val_accuracy: 0.4416 - val_loss: 1.2090
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 87s 3s/step - accuracy: 0.5566 - loss: 1.0966 - val_accuracy: 0.6169 - val_loss: 1.0048
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.6343 - loss: 0.8860 - val_accuracy: 0.4870 - val_loss: 1.2997
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.6642 - loss: 0.8289 - val_accuracy: 0.6299 - val_loss: 1.2837
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.7768 - loss: 0.6032 - val_accuracy: 0.6234 - val_loss: 1.1407


In [20]:
!pip install transformers datasets


In [21]:
from transformers import ViTFeatureExtractor
import tensorflow as tf

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Function to preprocess images for ViT
def preprocess(images, labels):
    images = tf.cast(images, tf.float32) / 255.0  # normalize
    # ViT expects RGB [0,1]
    return images, labels
train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [29]:
!pip install torch torchvision transformers timm


In [32]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

data_dir = "/content/Monkeypox_Skin_Image_Dataset/Monkeypox Skin Image Dataset"

# Transform for ViT: resize, normalize
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# Load dataset
full_dataset = datasets.ImageFolder(data_dir, transform=transform)
class_names = full_dataset.classes
print("Classes:", class_names)

# Train/Validation split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


Classes: ['Chickenpox', 'Measles', 'Monkeypox', 'Normal']


In [33]:
import torch
from transformers import ViTForImageClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(class_names)
)
model.to(device)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [34]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-5)  # correct AdamW location


In [35]:
from tqdm import tqdm

epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {correct/total:.4f}")


100%|██████████| 39/39 [24:14<00:00, 37.29s/it]


Epoch 1, Loss: 1.0032, Accuracy: 0.7094


100%|██████████| 39/39 [23:24<00:00, 36.01s/it]


Epoch 2, Loss: 0.4727, Accuracy: 0.9253


100%|██████████| 39/39 [23:32<00:00, 36.23s/it]


Epoch 3, Loss: 0.2667, Accuracy: 0.9627


100%|██████████| 39/39 [23:19<00:00, 35.87s/it]


Epoch 4, Loss: 0.1673, Accuracy: 0.9838


100%|██████████| 39/39 [23:16<00:00, 35.81s/it]

Epoch 5, Loss: 0.1098, Accuracy: 0.9951
